In [1]:
!git clone https://github.com/DngBack/capstone_l2r_ess.git

Cloning into 'capstone_l2r_ess'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 778 (delta 54), reused 87 (delta 45), pack-reused 662 (from 1)
Receiving objects: 100% (778/778), 27.14 MiB | 21.55 MiB/s, done.
Resolving deltas: 100% (270/270), done.


In [2]:
!cd /content/capstone_l2r_ess

/content/capstone_l2r_ess


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.0 MB/s eta 0:00:00


In [5]:
!mkdir data

In [6]:
!cd data

/content/capstone_l2r_ess/data


In [8]:
!tar -xvzf /content/capstone_l2r_ess/data/cifar-100-python.tar.gz

cifar-100-python/
cifar-100-python/file.txt~
cifar-100-python/train
cifar-100-python/test
cifar-100-python/meta


In [11]:
cd ..

/content/capstone_l2r_ess


In [12]:
!python src/data/balanced_test_splits.py

CREATING CIFAR-100-LT WITH BALANCED TEST SPLITS

Configuration:
  Imbalance Factor: 100
  Output Directory: data/cifar100_lt_if100_splits_fixed
  Random Seed: 42
  Train Split: 90% Expert / 10% Gating

STEP 1: Loading CIFAR-100 datasets...
  - Train: 50,000 samples
  - Test:  10,000 samples

STEP 2: Creating long-tail training set...
Creating CIFAR-100-LT training set (IF=100)...
  Total samples: 10,847
  Head class (0): 500 samples
  Tail class (99): 5 samples
  Imbalance Factor: 100.0

STEP 3: Splitting train set (Expert vs Gating)...

Splitting training set (90% expert / 10% gating)...

  SUCCESS: Expert split:
    Total: 9,719 samples
    Head class (0): 450 samples
    Tail class (99): 4 samples
    Imbalance Factor: 112.5

  SUCCESS: Gating split:
    Total: 1,128 samples
    Head class (0): 50 samples
    Tail class (99): 1 samples
    Imbalance Factor: 50.0

  SUCCESS: No overlap between expert and gating splits

  SUCCESS: Imbalance ratio preserved:
    Original: 100.0
    Exp

In [14]:
!python train_experts.py --dataset cifar100_lt_if100 --log-file logs/experts_cifar.log


LOGGING TO FILE: logs/experts_cifar.log
STARTED AT: 2025-12-04 10:12:48

AR-GSE EXPERT TRAINING
📊 Training Mode: Using FULL train set (default)
   - Trains on full train set (100% of train)
   - Validates on balanced val set
   - Uses reweighted metrics for validation

✓ Using dataset: cifar100_lt_if100
  Classes: 100
  Backbone: cifar_resnet32
  Batch size: 128
🚀 AR-GSE Expert Training Pipeline
Device: cpu
Dataset: cifar100_lt_if100
Experts to train: ['ce', 'logitadjust', 'balsoftmax']

🎯 Training Expert: CE

[EXPERT] TRAINING EXPERT: CE_BASELINE
[EXPERT] Loss Type: CE
[EXPERT] Splits Dir: ./data/cifar100_lt_if100_splits_fixed
Loading CIFAR-100-LT datasets...
  Using FULL train split for training
Setting up CIFAR-100-LT datasets...
  train: 10,847 samples
  val: 1,000 samples
  test: 8,000 samples
  tunev: 1,000 samples
Datasets setup complete!
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in to

In [15]:
!python -m src.train.train_gating_map --dataset cifar100_lt_if100 --routing dense

✓ Using dataset: cifar100_lt_if100
  Classes: 100
  Experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
TRAINING GATING NETWORK FOR MAP
Loading expert logits and labels...
  Loading gating split (10% of train with same long-tail)...
    Train: 1,128 samples, 3 experts, 100 classes
    Train logits shape: torch.Size([1128, 3, 100])
    Train labels shape: torch.Size([1128])
  Loading val split...
    Val: 1,000 samples
    Val logits shape: torch.Size([1000, 3, 100])
    Val labels shape: torch.Size([1000])
[OK] Loaded class weights (range: [0.0005, 0.0461])

Creating GatingNetwork:
   Experts: 3
   Classes: 100
   Routing: dense
   Hidden: [256, 128]
   Total params: 114,691
   Feature dim: 314

Loss Configuration:
   Mixture NLL: ✓
   Load-balancing: ✗ (disabled for dense routing)
   Entropy reg: ✓
   Responsibility loss: ✓
   Prior regularizer: ✓
   λ_H: 0.01
   λ_Resp: 0.1
   λ_Prior: 0.05

Estimating group priors from training data...
   Group priors shape: torc

In [16]:
!python run_balanced_plugin_gating.py --dataset cifar100_lt_if100

✓ Using dataset: cifar100_lt_if100
  Classes: 100
  Experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
Loading gating network...
Loading gating network for 3 experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
✓ Loaded gating network from: checkpoints/gating_map/cifar100_lt_if100/final_gating.pth
✓ Gating network configured for 3 experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
Loading S1 (tunev) and S2 (val) for selection/evaluation...
Loading logits for 3 experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
  ✓ Loaded ce_baseline: torch.Size([1000, 100])
  ✓ Loaded logitadjust_baseline: torch.Size([1000, 100])
  ✓ Loaded balsoftmax_baseline: torch.Size([1000, 100])
✓ Stacked expert logits: torch.Size([1000, 3, 100]) (should be [N, 3, 100])
Loading logits for 3 experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
  ✓ Loaded ce_baseline: torch.Size([1000, 100])
  ✓ Loaded logitadjust_b

In [17]:
!python run_worst_plugin_gating.py --dataset cifar100_lt_if100

✓ Using dataset: cifar100_lt_if100
  Classes: 100
  Experts: ['ce_baseline', 'logitadjust_baseline', 'balsoftmax_baseline']
/content/capstone_l2r_ess/run_worst_plugin_gating.py:722: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  float(np.trapz(ew, r)) if r.size > 1 else float(ew.mean() if ew.size else 0.0)
/content/capstone_l2r_ess/run_worst_plugin_gating.py:725: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  float(np.trapz(eb, r)) if r.size > 1 else float(eb.mean() if eb.size else 0.0)
Saved results to: results/ltr_plugin/cifar100_lt_if100/ltr_plugin_gating_worst.json
Saved plot to: results/ltr_plugin/cifar100_lt_if100/ltr_rc_curves_balanced_worst_gating_test.png
Saved gap plot to: results/ltr_plugin/cifar100_lt_if100/ltr_tail_minus_head_worst_gating_test.png
Saved CSV results to: results/ltr_plugin/cifar1